In [1]:
!pwd

/Users/raphaelmenguy/code/projet_wagon/elaine_and_the_gentle_boyz/notebooks


In [2]:
cd ..

/Users/raphaelmenguy/code/projet_wagon/elaine_and_the_gentle_boyz


In [3]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import joblib
import pandas as pd
from egbz.utils import *

In [4]:
PATH = 'data/'

# Données météo

source: https://www.data.gouv.fr/fr/datasets/donnees-quotidiennes-de-119-stations-en-france-metropolitaine-pour-les-etudes-de-liens-entre-meteorologie-et-covid-19-du-01-01-2020-au-21-04-2021/

In [7]:
meteo = pd.read_csv(PATH+'meteo-france.csv', sep=';')
dep = pd.read_csv(PATH+'name.csv')
dep['dep'] = dep.iloc[:, 0].astype('str').apply(lambda x: '0'+x if len(x)==1 else x)
dep = dep.groupby('dep').first().reset_index()[['dep', 'NOM']]
meteo = meteo.merge(dep, on='NOM')
meteo['date'] = pd.to_datetime(meteo.DATE, format='%Y%m%d')
meteo['TX'] = meteo['TX'].apply(lambda x: x.replace(',', '.')).astype('float')
meteo['RR'] = meteo['RR'].apply(lambda x: x.replace(',', '.')).astype('float')
meteo = meteo[['date', 'dep', 'TX', 'UX', 'RR']].dropna().drop_duplicates()
meteo
meteo.to_csv('data/meteo_Elaine.csv')

 - *TX*: Température maximale
 - *UX*: Humidité maximale
 - *RR*: Hauteur de précipitation

# Données de pollution et mobilités

source: Ludo

In [7]:
pollution_mobility = pd.read_csv(PATH+'pollution_mobility.csv')
pollution_mobility['date'] = pd.to_datetime(pollution_mobility.date)
pollution_mobility['dep'] = (pollution_mobility.numero
                                 .astype('str')
                                 .apply(lambda x: x if len(x)>1 else '0'+x)
                            )
pollution_mobility = pollution_mobility[['date', 'dep', 'pm25', 'pm10',
                                         'mobindex1', 'mobindex2']]
pollution_mobility = pollution_mobility.drop_duplicates().sort_values('dep')
index_201 = pollution_mobility[pollution_mobility['dep']=='201'].index
index_202 = pollution_mobility[pollution_mobility['dep']=='202'].index
pollution_mobility.drop(index_202, inplace=True)
pollution_mobility.at[index_201, 'dep'] = '20'

# Données hospitalières

source: https://www.data.gouv.fr/fr/datasets/synthese-des-indicateurs-de-suivi-de-lepidemie-covid-19/

Il y a pas mal de missing values à prendre en compte. cvdose1 est vide -> on le drop. Puis, regardons le R: il semble être donné de manière irrégulière à partir de juin 2020, on utilise un imputer temporel pour remplir les valeurs manquantes (remarquons qu'avant juillet le R semble être fixé arbitrairement à 1). Pour les taux, il faut également interpoler quelques valeurs et les faires commencer autour de juin

In [8]:
covid_dep = pd.read_csv(PATH+'covid_dep.csv')
covid_dep['date'] = pd.to_datetime(covid_dep.date)
covid_dep['dep'] = covid_dep['dep'].astype('str')
nom_dep = covid_dep[['lib_dep', 'dep']].drop_duplicates()
covid_dep = (
    covid_dep
        .drop(['reg', 'lib_dep', 'lib_reg', 'cv_dose1'], axis=1)
        .set_index('date')
        .interpolate(method='time')
        .dropna()
        .reset_index()
)
index_2A = covid_dep.query('dep=="2A"').index
index_2B = covid_dep.query('dep=="2B"').index
covid_dep.at[index_2A, 'dep'] = '20'
covid_dep.drop(index_2B, inplace=True)
covid_dep

/Users/raphaelmenguy/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,dep,tx_pos,tx_incid,TO,R,hosp,rea,rad,dchosp,reg_rea,incid_hosp,incid_rea,incid_rad,incid_dchosp,reg_incid_rea,pos,pos_7j
0,2020-06-12,01,1.063201,2.739914,0.080501,1.020000,64,0,409,101,45,0.0,0.0,2.0,1.0,0.0,4.0,18.0
1,2020-06-13,01,0.894988,2.283261,0.073345,1.293333,64,0,409,101,41,0.0,0.0,0.0,0.0,1.0,0.0,15.0
2,2020-06-14,01,0.892857,2.283261,0.073345,1.076667,64,0,409,101,41,0.0,0.0,0.0,0.0,0.0,1.0,15.0
3,2020-06-15,01,0.891266,2.283261,0.069767,0.940000,64,0,410,101,39,0.0,0.0,1.0,0.0,0.0,0.0,15.0
4,2020-06-16,01,0.999412,2.587696,0.067979,0.798000,65,0,410,101,38,2.0,0.0,0.0,0.0,1.0,3.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53338,2021-08-24,976,1.519937,27.552054,0.000000,1.100000,1,0,1275,132,0,0.0,0.0,0.0,0.0,0.0,11.0,77.0
53339,2021-08-25,976,1.545436,26.836416,0.000000,1.100000,1,0,1275,132,0,0.0,0.0,0.0,0.0,0.0,9.0,75.0
53340,2021-08-26,976,1.545436,26.836416,0.000000,1.100000,1,0,1276,132,0,1.0,0.0,1.0,0.0,0.0,9.0,75.0
53341,2021-08-27,976,1.545436,26.836416,0.000000,1.100000,2,0,1276,132,0,1.0,0.0,0.0,0.0,0.0,9.0,75.0


In [9]:
covid_fr = pd.read_csv(PATH+'covid_france.csv')
covid_fr['date'] = pd.to_datetime(covid_fr.date)
covid_fr = (
    covid_fr
        .set_index('date')
        .drop(['cv_dose1', 'esms_cas', 'conf_j1'], axis=1)
        .interpolate(method='time')
        .dropna()
        .reset_index()
)
covid_fr['zone'] = ['france' for _ in range(len(covid_fr))]
covid_fr

,date,tx_pos,tx_incid,TO,R,rea,hosp,rad,dchosp,incid_rea,incid_hosp,incid_rad,incid_dchosp,conf,pos,esms_dc,dc_tot,pos_7j,zone
0,2020-06-01,1.496918,5.167251,0.251087,0.830987,1270.0,14237.0,68431.0,18486.0,9.0,89.0,95.0,31.0,152091,171.0,10327.0,28813.0,3468.0,france
1,2020-06-02,1.508338,5.092752,0.241400,0.826380,1221.0,13978.0,68803.0,18570.0,33.0,229.0,402.0,85.0,151325,723.0,10350.0,28920.0,3418.0,france
2,2020-06-03,1.488887,4.988453,0.233096,0.826668,1179.0,13465.0,69446.0,18651.0,26.0,250.0,668.0,85.0,151677,659.0,10350.0,29001.0,3348.0,france
3,2020-06-04,1.500908,4.960143,0.224002,0.831327,1133.0,13054.0,69967.0,18695.0,23.0,195.0,544.0,46.0,152444,622.0,10350.0,29045.0,3329.0,france
4,2020-06-05,1.504178,4.907994,0.210558,0.839821,1065.0,12650.0,70494.0,18741.0,19.0,213.0,548.0,47.0,153055,560.0,10350.0,29091.0,3294.0,france
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,2021-08-24,2.963911,204.192819,0.439106,0.929007,2221.0,11066.0,404141.0,87143.0,214.0,1145.0,871.0,153.0,6649630,21226.0,26505.0,111587.0,137044.0,france
450,2021-08-25,2.920166,198.121150,0.442665,0.929007,2239.0,11171.0,404823.0,87236.0,182.0,883.0,701.0,93.0,6673336,18814.0,26505.0,111587.0,132969.0,france
451,2021-08-26,2.920166,198.121150,0.447015,0.929007,2261.0,11104.0,405490.0,87343.0,191.0,741.0,692.0,110.0,6693019,18814.0,26505.0,111587.0,132969.0,france
452,2021-08-27,2.920166,198.121150,0.448794,0.929007,2270.0,11162.0,406229.0,87438.0,194.0,908.0,750.0,95.0,6711268,18814.0,26505.0,111587.0,132969.0,france


# Variants

source: https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/

# Données démographiques

L'idée est d'avoir des données qui sont constantes dans le temps: en effet, il est impossible sinon d'inférer ce qui se passe dans un département sur lequel on se serait pas entraîné.

source: https://www.data.gouv.fr/fr/datasets/personnes-en-affection-de-longue-duree-ald/

Attention les données issues de Mayotte ne sont pas fiable (976). Pour avoir le détail des noms de colonnes, voir la source

In [10]:
mld = pd.read_excel(PATH+'mld_2019_dep.xls')
mld = (
    mld
        .dropna()
        .drop(['Nom du département', 'Bilharziose compliquée (ALD4)'], axis=1)
)
mld.columns = [x[-6:].strip('()') for x in mld.columns]
# Fusionne Corse du nord et du sud en une ligne
mld.iloc[19, :] = mld.iloc[20, :] + mld.iloc[19, :]
mld.iloc[19, 0] = '20'
mld.drop([20, 100], inplace=True)
mld.iloc[:, 1:] = mld.iloc[:, 1:].astype('float')
mld

,dep,ALD1,ALD2,ALD3,ALD5,ALD6,ALD7,ALD8,ALD9,ALD10,...,ALD21,ALD22,ALD23,ALD24,ALD25,ALD26,ALD27,ALD28,ALD29,ALD30
0,01,814.0,47.0,841.0,1963.0,245.0,106.0,4565.0,448.0,17.0,...,211.0,429.0,2122.0,247.0,137.0,121.0,216.0,25.0,22.0,3873.0
1,02,782.0,45.0,1003.0,2199.0,223.0,94.0,6284.0,618.0,20.0,...,178.0,406.0,2160.0,454.0,198.0,45.0,244.0,27.0,11.0,3718.0
2,03,918.0,52.0,1090.0,2151.0,262.0,130.0,5169.0,765.0,17.0,...,219.0,392.0,3503.0,272.0,204.0,211.0,273.0,37.0,14.0,4353.0
3,04,804.0,54.0,1110.0,2065.0,317.0,182.0,4135.0,741.0,26.0,...,258.0,469.0,3540.0,297.0,137.0,82.0,449.0,25.0,18.0,3887.0
4,05,867.0,64.0,1058.0,2367.0,331.0,149.0,3490.0,794.0,20.0,...,312.0,600.0,3462.0,327.0,158.0,82.0,431.0,31.0,11.0,3945.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,729.0,50.0,743.0,1713.0,330.0,306.0,5528.0,539.0,79.0,...,252.0,331.0,1982.0,263.0,136.0,71.0,229.0,27.0,44.0,3490.0
96,971,1035.0,26.0,552.0,1302.0,216.0,415.0,8108.0,661.0,198.0,...,202.0,309.0,2291.0,99.0,41.0,90.0,94.0,18.0,25.0,2735.0
97,972,893.0,24.0,764.0,1314.0,146.0,273.0,7097.0,592.0,229.0,...,243.0,375.0,1841.0,142.0,59.0,97.0,87.0,13.0,25.0,2655.0
98,973,1136.0,18.0,355.0,1689.0,348.0,1037.0,7163.0,472.0,219.0,...,159.0,252.0,1001.0,64.0,26.0,28.0,54.0,18.0,73.0,2096.0


source: Insee

In [11]:
li = pd.read_csv(PATH+'low_income.csv', sep=';')
li = (li
    .iloc[:, :-9]
    .T
)
li.drop(['Critère']+[x for x in li.index if x.upper()==x], inplace=True)
li['lib_dep'] = li.index
li = li.merge(nom_dep)[['dep', 0, 1]]
li.columns = ['dep', 'tpauv', 'talloc']
li['tpauv'] = li['tpauv'].apply(lambda x:x.replace(',','.')).astype(float)
li['talloc'] = li['talloc'].apply(lambda x:x.replace(',','.')).astype(float)
deps = [str(x) if x>9 else '0'+str(x) for x in range(1, 95)]
li = (pd.DataFrame({'dep': deps})
    .merge(li, on='dep', how='outer')
    .sort_values('dep')
    .fillna(li[['tpauv', 'talloc']].mean())
     )
li

,dep,tpauv,talloc
0,01,10.7,2.3
1,02,18.9,7.5
2,03,15.5,6.1
3,04,16.4,5.0
4,05,14.3,3.6
...,...,...,...
90,91,12.9,4.4
91,92,12.2,3.4
92,93,28.6,10.6
93,94,16.7,6.0


Source: Ludo

In [12]:
fumeurs = pd.read_csv(PATH+'fumeurs.csv', sep=';')
fumeurs['dep'] = (fumeurs.depnum
                         .astype('str')
                         .apply(lambda x: x if len(x)>1 else '0'+x)
                     )
fumeurs = fumeurs[['dep', 'Smokers']].sort_values('dep').reset_index(drop=True)
fumeurs.columns = ['dep', 'smok']
# Fusionne Corse du nord et du sud en une ligne
fumeurs.iloc[19, 1] = 0.5*(fumeurs.iloc[20, 1] + fumeurs.iloc[19, 1])
fumeurs.iloc[19, 0] = '20'
fumeurs.drop(20, inplace=True)
fumeurs

,dep,smok
0,01,0.262
1,02,0.305
2,03,0.262
3,04,0.321
4,05,0.321
...,...,...
91,91,0.213
92,92,0.213
93,93,0.213
94,94,0.213


# Dataset

In [13]:
constant_dataset = xr.merge([
    create_dataset(li, zone_col='dep'),
    create_dataset(fumeurs, zone_col='dep'),
    create_dataset(mld, zone_col='dep')
])
constant_dataset

<xarray.Dataset>
Dimensions:  (zone: 99)
Coordinates:
  * zone     (zone) object '01' '02' '03' '04' '05' ... '971' '972' '973' '974'
Data variables: (12/32)
    tpauv    (zone) float64 10.7 18.9 15.5 16.4 14.3 ... 16.8 nan nan nan nan
    talloc   (zone) float64 2.3 7.5 6.1 5.0 3.6 4.2 ... 6.0 5.8 nan nan nan nan
    smok     (zone) float64 0.262 0.305 0.262 0.321 0.321 ... nan nan nan nan
    ALD1     (zone) float64 814.0 782.0 918.0 ... 893.0 1.136e+03 1.132e+03
    ALD2     (zone) float64 47.0 45.0 52.0 54.0 64.0 ... 26.0 24.0 18.0 33.0
    ALD3     (zone) float64 841.0 1.003e+03 1.09e+03 ... 764.0 355.0 630.0
    ...       ...
    ALD25    (zone) float64 137.0 198.0 204.0 137.0 ... 41.0 59.0 26.0 26.0
    ALD26    (zone) float64 121.0 45.0 211.0 82.0 82.0 ... 90.0 97.0 28.0 24.0
    ALD27    (zone) float64 216.0 244.0 273.0 449.0 ... 94.0 87.0 54.0 166.0
    ALD28    (zone) float64 25.0 27.0 37.0 25.0 31.0 ... 18.0 13.0 18.0 21.0
    ALD29    (zone) float64 22.0 11.0 14.0 18.0 11.0 ... 25.0 25.0 73.0 22.0
    ALD30    (zone) float64 3.873e+03 3.718e+03 ... 2.096e+03 2.135e+03

In [18]:
covid_fr

,date,tx_pos,tx_incid,TO,R,rea,hosp,rad,dchosp,incid_rea,incid_hosp,incid_rad,incid_dchosp,conf,pos,esms_dc,dc_tot,pos_7j,zone
0,2020-06-01,1.496918,5.167251,0.251087,0.830987,1270.0,14237.0,68431.0,18486.0,9.0,89.0,95.0,31.0,152091,171.0,10327.0,28813.0,3468.0,france
1,2020-06-02,1.508338,5.092752,0.241400,0.826380,1221.0,13978.0,68803.0,18570.0,33.0,229.0,402.0,85.0,151325,723.0,10350.0,28920.0,3418.0,france
2,2020-06-03,1.488887,4.988453,0.233096,0.826668,1179.0,13465.0,69446.0,18651.0,26.0,250.0,668.0,85.0,151677,659.0,10350.0,29001.0,3348.0,france
3,2020-06-04,1.500908,4.960143,0.224002,0.831327,1133.0,13054.0,69967.0,18695.0,23.0,195.0,544.0,46.0,152444,622.0,10350.0,29045.0,3329.0,france
4,2020-06-05,1.504178,4.907994,0.210558,0.839821,1065.0,12650.0,70494.0,18741.0,19.0,213.0,548.0,47.0,153055,560.0,10350.0,29091.0,3294.0,france
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,2021-08-24,2.963911,204.192819,0.439106,0.929007,2221.0,11066.0,404141.0,87143.0,214.0,1145.0,871.0,153.0,6649630,21226.0,26505.0,111587.0,137044.0,france
450,2021-08-25,2.920166,198.121150,0.442665,0.929007,2239.0,11171.0,404823.0,87236.0,182.0,883.0,701.0,93.0,6673336,18814.0,26505.0,111587.0,132969.0,france
451,2021-08-26,2.920166,198.121150,0.447015,0.929007,2261.0,11104.0,405490.0,87343.0,191.0,741.0,692.0,110.0,6693019,18814.0,26505.0,111587.0,132969.0,france
452,2021-08-27,2.920166,198.121150,0.448794,0.929007,2270.0,11162.0,406229.0,87438.0,194.0,908.0,750.0,95.0,6711268,18814.0,26505.0,111587.0,132969.0,france


In [14]:
time_dataset = xr.merge([
    create_dataset(meteo, zone_col='dep', time_col='date'),
    create_dataset(pollution_mobility, zone_col='dep', time_col='date'),
    create_dataset(covid_dep, zone_col='dep', time_col='date'),
    create_dataset(covid_fr, zone_col='zone', time_col='date'),
])
time_dataset

<xarray.Dataset>
Dimensions:        (time: 606, zone: 101)
Coordinates:
  * time           (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2021-08-28
  * zone           (zone) object '01' '02' '03' '04' ... '974' '976' 'france'
Data variables: (12/26)
    TX             (zone, time) float64 0.0 8.7 9.1 8.7 4.9 ... nan nan nan nan
    UX             (zone, time) float64 98.0 99.0 95.0 97.0 ... nan nan nan nan
    RR             (zone, time) float64 0.0 0.0 0.4 0.6 0.0 ... nan nan nan nan
    pm25           (zone, time) float64 nan nan nan nan nan ... nan nan nan nan
    pm10           (zone, time) float64 nan nan nan nan nan ... nan nan nan nan
    mobindex1      (zone, time) float64 nan nan nan nan nan ... nan nan nan nan
    ...             ...
    reg_incid_rea  (zone, time) float64 nan nan nan nan nan ... nan nan nan nan
    pos            (zone, time) float64 nan nan nan ... 1.881e+04 1.881e+04
    pos_7j         (zone, time) float64 nan nan nan ... 1.33e+05 1.33e+05
    conf           (zone, time) float64 nan nan nan ... 6.711e+06 6.729e+06
    esms_dc        (zone, time) float64 nan nan nan ... 2.650e+04 2.650e+04
    dc_tot         (zone, time) float64 nan nan nan ... 1.116e+05 1.116e+05

In [15]:
dataset = xr.merge([constant_dataset, time_dataset])
dataset

<xarray.Dataset>
Dimensions:        (time: 606, zone: 101)
Coordinates:
  * zone           (zone) object '01' '02' '03' '04' ... '974' '976' 'france'
  * time           (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2021-08-28
Data variables: (12/58)
    tpauv          (zone) float64 10.7 18.9 15.5 16.4 14.3 ... nan nan nan nan
    talloc         (zone) float64 2.3 7.5 6.1 5.0 3.6 ... nan nan nan nan nan
    smok           (zone) float64 0.262 0.305 0.262 0.321 ... nan nan nan nan
    ALD1           (zone) float64 814.0 782.0 918.0 804.0 ... 1.132e+03 nan nan
    ALD2           (zone) float64 47.0 45.0 52.0 54.0 64.0 ... 18.0 33.0 nan nan
    ALD3           (zone) float64 841.0 1.003e+03 1.09e+03 ... 630.0 nan nan
    ...             ...
    reg_incid_rea  (zone, time) float64 nan nan nan nan nan ... nan nan nan nan
    pos            (zone, time) float64 nan nan nan ... 1.881e+04 1.881e+04
    pos_7j         (zone, time) float64 nan nan nan ... 1.33e+05 1.33e+05
    conf           (zone, time) float64 nan nan nan ... 6.711e+06 6.729e+06
    esms_dc        (zone, time) float64 nan nan nan ... 2.650e+04 2.650e+04
    dc_tot         (zone, time) float64 nan nan nan ... 1.116e+05 1.116e+05

In [17]:
joblib.dump(dataset, 'training_data/dataset.pkl')
joblib.dump(constant_dataset, 'training_data/c_dataset.pkl')
joblib.dump(time_dataset, 'training_data/t_dataset.pkl')

['training_data/t_dataset.pkl']